# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

from queries import *

In [ ]:
# checking your current working directory
print(os.getcwd())

# Getting current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    
# joining the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(filepath+'/*.csv')
    print(len(file_path_list))
    break
file_path_list.sort()

###### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
    print(f)
# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
#creating combined csv file
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))
        

In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II ETL Pipeline for creating and quering Database 

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## A. Create Connection to the Database

#### Creat Cluster

In [ ]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

session = cluster.connect()

#### Create Keyspace

In [ ]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project2
    WITH REPLICATION=
    {
    'class':'SimpleStrategy',
    'replication_factor':1
    }""")
except Exception as e:
    print(e)

#### Set Keyspace

In [ ]:
try:
    session.set_keyspace('project2')
except Exception as e:
    print(e)

## B. Create Denormalized Tables

### B1. Music_Library Table
**Query Example:** <br> Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

#### CREATE MUSIC_LIBRARY TABLE

* The music_library table will be partitioned by sessionId that will be the partition key, and itemInSession will be the clustering colum to make each Primary Key unique. 

In [ ]:
session.execute(music_library_drop)
try:
    session.execute(create_music_library_table)
except Exception as e:
    print(e)                   

#### INSERT INTO MUSIC_LIBRARY TABLE

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = music_library_insert
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])) )

#### SELECT FROM MUSIC_LIBRARY TABLE

**Selecting the specified set of values according to the query**

In [ ]:
select_query =select_from_music_library

try:
    rows = session.execute(select_query, (338, 4))
except Exception as e:
    print(e)

result=[]
for row in rows:
    result.append(row)
    df=pd.DataFrame(result)
df

### B2. User_Library Table
**Query Example:** <br> Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### CREATE USER_LIBRARY TABLE

* The user_library table will be partitioned by userid and sessionid that will be the partition key(composite primary key), and  itemInSession will be the clustering colum to make each Primary Key unique ans sorting the items in the table. 

In [ ]:
session.execute(user_library_drop)

#creating the table
create_query = create_user_library_table

# executing create_query
try:
    session.execute(create_query)
except Exception as e:
    print(e)              



#### INSERT INTO USER_LIBRARY TABLE

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:

        insert_query = user_table_insert
        
        session.execute(insert_query, (line[0], line[9], int(line[3]), line[1], line[4], int(line[10]), int(line[8])))
                    

#### SELECT FROM USER_LIBRARY TABLE

**Selecting the specified set of values according to the query**

In [15]:
select_query = select_from_user_library
try:
    rows = session.execute(select_query, (10, 182))
except Exception as e:
    print(e)

result=[]
for row in rows:
    result.append(row)
    df=pd.DataFrame(result)
df

artist                                               song  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

   iteminsession firstname lastname  
0              0    Sylvie     Cruz  
1              1    Sylvie     Cruz  
2              2    Sylvie     Cruz  
3              3    Sylvie     Cruz

### B3. Song_Library Table
**Query Example:** <br> Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### CREATE SONG_LIBRARY TABLE

* The song_library table will be partitioned by song that will be the partition key and  userid will be the clustering colum to make each Primary Key unique. 

In [16]:
session.execute(song_library_drop)

# sql-query for creating the table
create_query = create_song_library_table

# executing create_query
try:
    session.execute(create_query)
except Exception as e:
    print(e)                   

#### INSERT INTO SONG_LIBRARY TABLE

In [17]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        insert_query =song_table_insert     
        session.execute(insert_query, (line[1], line[4], int(line[10]), line[9]))

#### SELECT FROM SONG_LIBRARY TABLE

**Selecting the specified set of values according to the query**

In [19]:
select_query =select_from_song_table

try:
    rows = session.execute(select_query, ('All Hands Against His Own',))
except Exception as e:
    print(e)

result=[]
for row in rows:
    result.append(row)
    df=pd.DataFrame(result)
df

firstname lastname  userid                       song
0  Jacqueline    Lynch      29  All Hands Against His Own
1       Tegan   Levine      80  All Hands Against His Own
2        Sara  Johnson      95  All Hands Against His Own

#### DROP TABLES

In [20]:
query = music_library_drop
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = user_library_drop
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = song_library_drop
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()